# 对齐 Query 和文档

在 RAG 系统中，用户的原始查询可能因措辞不精确、缺乏语义信息或包含指代词等原因，导致检索结果不符合预期。因此，将用户查询的语义空间与文档的语义空间保持一致至关重要。

本节旨在介绍主要的 query 对齐技术：
1. **Query改写**：优化查询表述
2. **HyDE（假设文档嵌入）**：生成假设文档后检索
3. **Step-Back Prompting**：生成抽象问题后检索
4. **子查询（MultiQueryRetriever）**：生成多个子查询后检索


## 第0章：数据准备与评估框架

本章准备实验所需的数据和评估工具。

### 0.1 环境准备

In [ ]:
# 导入必要的库
import re
import json
import pandas as pd
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.chat_models import ChatZhipuAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents.base import Document
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from tqdm import tqdm
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

# 加载环境变量
load_dotenv()
os.environ['ZHIPUAI_API_KEY'] = os.getenv('ZHIPUAI_API_KEY')

### 0.2 数据加载

In [ ]:
# 数据路径（相对于当前notebook所在目录）
pdf_path = "../3. 索引阶段/data/pumpkin_book.pdf"
qa_path = '../3. 索引阶段/data/train_dataset.json'

# 初始化 LLM
llm = ChatZhipuAI(model="glm-4-flash", temperature=0.0, timeout=1200, max_retries=3, api_key=os.environ["ZHIPUAI_API_KEY"])

# 初始化 Embedding 模型
embedding = ZhipuAIEmbeddings(
    api_key=os.environ['ZHIPUAI_API_KEY'],
    model="embedding-3-pro" 
)
print("✅ Embedding模型初始化成功")

# 加载问答数据集
with open(qa_path, 'r', encoding='utf-8') as file:
    qa_pairs = json.load(file)

print(f"✅ 加载了 {len(qa_pairs)} 个问答对")
print(f"问答对示例: {qa_pairs[0]}")

✅ Embedding模型初始化成功
✅ 加载了 119 个问答对
问答对示例: {'query': '请根据提供的上下文信息，解释“算法”和“模型”的概念，并说明它们在机器学习中的关系。', 'answer': '“算法”是指从数据中学得“模型”的具体方法，例如后续章节中将会讲述的线性回归、对数几率回归、决策树等。“算法”产出的结果称为“模型”，通常是具体的函数或者可抽象地看作为函数，例如一元线性回归算法产出的模型即为形如f(x)=wx+b的一元一次函数。不过由于严格区分这两者的意义不大，因此多数文献和资料会将其混用，当遇到这两个概念时，其具体指代根据上下文判断即可。', 'page_num': 13}


### 0.3 构建向量数据库

In [4]:
def clean_text(text: str):
    """
    实现文本清理函数（与CCH文档保持完全一致）
    
    参数:
        text: 需要清理的字段
    
    返回:
        清理完成后返回的字段
    """
    # 删除每页开头与结尾标语及链接
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    # 删除字符串开头的空格
    text = re.sub(r'\s+', '', text)
    # 删除回车
    text = re.sub(r'\n+', '', text)
    
    return text

def build_vectorstore(pdf_path, embedding, chunk_size=2000, chunk_overlap=100, persist_directory="./chroma_db"):
    """
    构建或加载向量数据库
    
    参数:
        persist_directory: 向量库持久化保存的目录
    """
    # 1. 检查是否已存在持久化的向量库
    if os.path.exists(persist_directory) and os.listdir(persist_directory):
        print(f"发现已存在的向量数据库: {persist_directory}，正在加载...")
        try:
            vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)
            count = vectorstore._collection.count() 
            print(f"✅ 加载成功！库中包含 {count} 个文档块。")
            return vectorstore
        except Exception as e:
            print(f"⚠️ 加载失败 ({e})，将重新构建...")
    
    # 2. 如果不存在，则重新构建
    print("未发现现有向量库或加载失败，开始重新构建...")
    
    loader = PyMuPDFLoader(pdf_path)
    pdf_pages = loader.load()
    data_pages = pdf_pages[13:-13]
    for page in data_pages:
        page.page_content = clean_text(page.page_content)
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator='')
    split_docs = text_splitter.split_documents(data_pages)
    for split_doc in tqdm(split_docs, desc="处理文档"):
        split_doc.metadata['chunk_size'] = chunk_size
        split_doc.metadata['chunk_overlap'] = chunk_overlap
    
    # 批处理构建向量数据库（避免 API限制）
    batch_size = 16
    vectorstore = None
    print(f"共{len(split_docs)}个文档块，将分{(len(split_docs) + batch_size - 1) // batch_size}批处理...")
    
    for i in tqdm(range(0, len(split_docs), batch_size), desc="批量构建向量库"):
        batch = split_docs[i:i+batch_size]
        if vectorstore is None:
            vectorstore = Chroma.from_documents(
                documents=batch, 
                embedding=embedding,
                persist_directory=persist_directory
            )
        else:
            vectorstore.add_documents(documents=batch)
    
    print(f"✅ 向量数据库构建完成并保存至 {persist_directory}！")
    return vectorstore

print("正在初始化向量数据库...")

vectorstore = build_vectorstore(pdf_path, embedding)

正在初始化向量数据库...
发现已存在的向量数据库: ./chroma_db，正在加载...
✅ 加载成功！库中包含 170 个文档块。


为了后续方便观察，我们选一些当前检索效果不佳的案例，进行后续的演示。
我们构建一个混合测试集：
1. Hard Cases: Baseline 检索失败的案例（主要关注点）
2. Easy Cases: Baseline 检索成功的案例（用于检测副作用）

In [ ]:
hard_indices = []
easy_indices = []

print(f"正在筛选测试用例...")

for idx in tqdm(range(len(qa_pairs)), desc="Baseline 扫描"):
    qa = qa_pairs[idx]
    query, target = qa['query'], qa['page_num']
    if not query.strip(): continue

    # 运行 Baseline
    docs = vectorstore.similarity_search(query, k=3)
    hit = target in [d.metadata['page'] for d in docs]
    
    if not hit:
        hard_indices.append(idx)
    else:
        easy_indices.append(idx)

# 打印筛选结果
print(f"\n筛选完成：")
print(f"❌ Hard Cases (Baseline 失败): {len(hard_indices)} 个")
print(f"✅ Easy Cases (Baseline 成功): {len(easy_indices)} 个")

# 构建最终测试集：全部 Hard Case + 随机采样的 Easy Case (例如 5 个)
# 这样既能突出优化效果，也能暴露潜在的副作用（Regression）
import random
random.seed(42)
sampled_easy = random.sample(easy_indices, min(5, len(easy_indices)))
test_indices = sorted(hard_indices + sampled_easy)

print(f"\n最终测试集包含 {len(test_indices)} 个样本 (含 {len(sampled_easy)} 个 Easy Case 用于检测副作用)。")


正在筛选测试用例...


Baseline 扫描: 100%|██████████| 119/119 [00:37<00:00,  3.17it/s]


筛选完成：
❌ Hard Cases (Baseline 失败): 6 个
✅ Easy Cases (Baseline 成功): 104 个

最终测试集包含 11 个样本 (含 5 个 Easy Case 用于检测副作用)。


### 0.4 定义通用评估函数

为了公平对比不同检索策略的效果，我们定义统一的评估与展示函数。

In [31]:
from typing import List, Dict, Callable, Tuple, Any
from tqdm import tqdm
import re

def evaluate_retrieval_method(
    method_name: str, test_indices: List[int], qa_pairs: List[Dict],
    vectorstore, method_func: Callable[[str], Tuple[List[Any], str]], topk: int = 3
):
    """
    评估指定检索方法的性能，并与 Baseline 进行对比。
    计算并展示 Recall@K 指标，以及相对于 Baseline 的提升幅度。

    Args:
        method_name (str): 待评估方法的名称（用于显示）。
        test_indices (List[int]): 测试用例在 qa_pairs 中的索引列表。
        qa_pairs (List[Dict]): 包含所有问答对的数据集。
        vectorstore: 向量数据库对象，用于执行 Baseline 检索。
        method_func (Callable): 待评估的检索函数。
            输入: query (str)
            输出: (documents, debug_info)
        topk (int, optional): 检索召回的文档数量. Defaults to 3.

    Returns:
        List[Dict]: 评估结果列表，包含每个测试用例的详细信息（Query, Answer, Hit情况, Debug信息等）。
    """
    results = []
    
    # 遍历测试集进行评估
    for idx in tqdm(test_indices, desc=f"{method_name} Testing"):
        qa, target = qa_pairs[idx], qa_pairs[idx]['page_num']
        
        # 1. 运行 Baseline (直接检索)
        b_docs = vectorstore.similarity_search(qa['query'], k=topk)
        # 检查 Baseline 是否命中目标页码
        b_hit = target in [d.metadata['page'] for d in b_docs]
        
        try:
            # 2. 运行待评估方法 (Method)
            h_docs, debug_info = method_func(qa['query'])
            # 检查待评估方法是否命中目标页码
            h_hit = target in [d.metadata['page'] for d in h_docs]
            
            # 记录结果
            results.append({
                'index': idx, 
                'query': qa['query'], 
                'target_page': target, 
                'answer': qa['answer'],
                'b_hit': b_hit, 
                'h_hit': h_hit,
                'h_docs': h_docs, 
                'debug_info': debug_info
            })
        except Exception as e:
            print(f"❌ Index {idx} Error: {e}")

    # 3. 计算并展示宏观指标
    total = len(results)
    if total == 0: return []
    
    # 计算召回率 (Recall)
    b_recall = sum(1 for r in results if r['b_hit']) / total
    h_recall = sum(1 for r in results if r['h_hit']) / total
    
    print(f"\n{'='*15} 📊 评估总结 (Summary) {'='*15}")
    print(f"测试样本数: {total}")
    print(f"Baseline Recall@{topk}: {b_recall:.1%}")
    print(f"{method_name:<8} Recall@{topk}: {h_recall:.1%} (提升: {h_recall - b_recall:+.1%})")
    print("="*48)
    
    return results

def display_cases(results: List[Dict], method_name: str, case_type: str = "win"):
    """
    展示特定类型的评估案例（Win Cases 或 Bad Cases），用于定性分析方法的效果。

    Args:
        results (List[Dict]): `evaluate_retrieval_method` 返回的评估结果列表。
        method_name (str): 方法名称。
        case_type (str, optional): 展示的案例类型. 
            - "win": 展示 Baseline 失败但该方法成功的案例（优化收益），用于分析优化效果。
            - "loss": 展示 Baseline 成功但该方法失败的案例（丢失案例），用于分析负作用。
            Defaults to "win".
    """
    # 根据类型筛选案例
    if case_type == "win":
        cases = [r for r in results if r['h_hit'] and not r['b_hit']]
        title = f"✅ {method_name} 优化收益 (Good Cases)"
    elif case_type == "loss":
        cases = [r for r in results if not r['h_hit'] and r['b_hit']]
        title = f"⚠️ {method_name} 丢失案例 (Bad Cases)"
    else:
        return

    # 如果没有符合条件的案例，直接返回
    if not cases: return print(f"\n未发现 {title}")
    
    print(f"\n{'='*30}\n{title}\n{'='*30}")
    
    # 逐个展示案例详情
    for i, res in enumerate(cases, 1):
        print(f"Case {i} | Index: {res['index']} | Target: P{res['target_page']}")
        print(f"【Query】: {res['query']}")
        print(f"【Answer】: {res['answer'][:100]}...")
        
        # 展示中间过程 (Debug Info)
        dbg = res['debug_info']
        print(f"【Process】:")
        if isinstance(dbg, (list, tuple)):
            # 如果是列表（如生成的多个子查询），逐行打印
            for j, q in enumerate(dbg, 1):
                clean_q = re.sub(r'^\d+\.\s*', '', str(q)).replace('\n', ' ')
                print(f"   {j}. {clean_q[:80]}...")
        else:
            # 如果是字符串，清理换行符后打印
            clean_dbg = str(dbg).replace('\n', ' ')
            print(f"   {clean_dbg[:200]}...")
            
        # 展示召回的文档 (Top-3)
        print(f"【Docs (Top-3)】:")
        for j, doc in enumerate(res.get('h_docs', [])[:3], 1):
            # 标记是否命中目标页面
            mark = "✅" if doc.metadata.get("page") == res["target_page"] else "  "
            content = doc.page_content[:80].replace('\n', ' ')
            print(f"  {mark} Doc {j} (P{doc.metadata.get('page')}): {content}...")
        print("-" * 60)

## 第1章：Query 改写

### 1.1 理论介绍

#### 1.1.1 定义
通过优化原始 query 的表述，使其更适合检索任务，解决用户原始 query 表达模糊或语义信息缺失的问题，从而提高检索的准确性和召回率。

#### 1.1.2 实现方法
实现 query 改写的主要方法有：
* **使用语言模型改写 query**：利用大语言模型对原始 query 进行改写，使其更加清晰、准确。例如，将复杂的自然语言问题转化为更简洁、更符合检索需求的表述。例如，将“我想知道关于人工智能的最新研究进展”改写为“人工智能最新研究论文”。
* **结合上下文信息改写 query**：在多轮对话中，结合之前的对话内容对当前 query 进行改写，使当前 query 的语义信息更完整。例如，在用户询问“详细介绍这个算法的实现原理”时，结合历史对话提到的算法A，替换句中的指代对象，改写为“算法A的实现原理”。

### 1.2 几种典型改写场景

下面通过实际例子演示几种常见的改写场景：

| 场景 | 问题 | 示例 |
|------|------|------|
| **口语化表达规范化** | 口语、网络用语难以匹配正式文档 | "公司报销那个APP咋用啊" → "公司报销APP使用教程" |
| **冗长查询精炼化** | 包含大量无关信息，核心不明确 | "我最近睡眠不好...想了解下那个是什么" → "睡眠监测手环推荐" |
| **拼写错误纠正** | 修正输入中的错别字或同音词 | "请假条的格试" → "请假条格式" |
| **术语缩写扩展** | 将英文缩写扩展为完整定义 | "HRBP是干嘛的" → "人力资源业务合作伙伴(HRBP)岗位职责" |
| **符号描述语义化** | 将符号的口语描述转为标准术语 | "微积分公式里那个倒三角符号是什么意思" → "微积分梯度算子(Nabla/∇)含义" |


### 1.3 实现改写链



In [7]:
# 定义改写prompt
rewrite_system_prompt = (
    "请将用户的查询改写为更清晰、更适合检索的形式。"
    "要求：只输出改写后的查询，不要输出任何分析、解释或多个选项。"
)

rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", rewrite_system_prompt),
    ("human", "{query}"),
])

# 创建改写链
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

print("=" * 80)
print("【Query改写场景演示】")
print("=" * 80)

# 定义测试场景
# 定义测试场景
# 定义测试场景
test_scenarios = [
    ("口语化表达规范化", "公司报销那个APP咋用啊"),
    ("冗长查询精炼化", "我最近睡眠不好，想找个那种能戴在手上的，能看睡了多久的东西，想了解下那个是什么"),
    ("拼写错误纠正", "请假条的格试"),
    ("术语缩写扩展", "HRBP是干嘛的"),
    ("符号描述语义化", "微积分公式里那个倒三角符号是什么意思")
]

# 逐个场景进行改写演示
for i, (scenario, query) in enumerate(test_scenarios, 1):
    print(f"\n【场景 {i}：{scenario}】")
    print(f"原始查询：{query}")
    
    try:
        # 调用LLM进行改写
        rewritten = rewrite_chain.invoke({"query": query})
        print(f"改写结果：{rewritten}")
    except Exception as e:
        print(f"改写失败：{e}")

【Query改写场景演示】

【场景 1：口语化表达规范化】
原始查询：公司报销那个APP咋用啊
改写结果：公司报销APP使用教程

【场景 2：冗长查询精炼化】
原始查询：我最近睡眠不好，想找个那种能戴在手上的，能看睡了多久的东西，想了解下那个是什么
改写结果：手环式睡眠监测器推荐

【场景 3：拼写错误纠正】
原始查询：请假条的格试
改写结果：请假条格式

【场景 4：术语缩写扩展】
原始查询：HRBP是干嘛的
改写结果：人力资源业务合作伙伴是做什么的

【场景 5：符号描述语义化】
原始查询：微积分公式里那个倒三角符号是什么意思
改写结果：微积分公式中的倒三角符号表示反正切函数，即arctan。


可以看出，改写后的查询更加准确，更加便于检索。

### 1.4 效果评估与对比

为了验证 Query 改写在真实检索场景中的效果，我们使用同样的评估框架，对比“原始 Query”与“改写后 Query”的检索表现。

In [ ]:
# 封装改写检索函数
basic_rewrite_chain = rewrite_prompt | llm | StrOutputParser()

def run_rewrite(query: str):
    # 1. 执行改写
    rewritten_query = basic_rewrite_chain.invoke({"query": query})
    
    # 2. 使用改写后的 Query 进行检索
    docs = vectorstore.similarity_search(rewritten_query, k=3)
    
    return docs, rewritten_query

# 执行评估
rewrite_eval_results = evaluate_retrieval_method(
    method_name="Query Rewrite",
    test_indices=test_indices,
    qa_pairs=qa_pairs,
    vectorstore=vectorstore,
    method_func=run_rewrite
)

Query Rewrite Testing: 100%|██████████| 11/11 [00:40<00:00,  3.71s/it]


=============== 📊 评估总结 (Summary) ===============
测试样本数: 11
Baseline Recall@3: 45.5%
Query Rewrite Recall@3: 54.5% (提升: +9.1%)


In [32]:
# 展示优化效果
display_cases(rewrite_eval_results, "Query Rewrite", case_type="win")

# 分析副作用
display_cases(rewrite_eval_results, "Query Rewrite", case_type="loss")


✅ Query Rewrite 优化收益 (Good Cases)
Case 1 | Index: 58 | Target: P103
【Query】: 解释8.5.13节中提到的a+bm·a+cm与am之间的区别，并阐述为什么在集成学习中，单独考虑各个分类器的概率（如a+bm·a+cm）与同时考虑它们的联合概率（如am）是重要的。
【Answer】: 在8.5.13节中，式(8.42)被拆分为更容易理解的形式，其中提到a+bm·a+cm为分类器hi与hj将样本预测为+1的概率的乘积，它表示的是分类器hi与hj独立预测的概率。而am表示的是分类器hi...
【Process】:
   8.5.13节中提到的a+bm·a+cm与am的区别，以及为什么在集成学习中单独考虑各个分类器的概率与同时考虑它们的联合概率是重要的。...
【Docs (Top-3)】:
     Doc 1 (P88): 8.1个体与集成基学习器(baselearner)的概念在论文中经常出现，可留意一下；另外，本节提到的投票法有两种，除了本节的多数投票(majorityvoti...
  ✅ Doc 2 (P103): 8.5.13式(8.42)的解释将式(8.42)拆分为如下形式，将会很容易理解其含义：p2=a+bm·a+cm+c+dm·b+dm其中a+bm为分类器hi将样本...
     Doc 3 (P102): RA(hi|x)p(x)dx表示个体学习器在全样本上的“分歧”，PTi=1wiRA(hi|x)p(x)dx表示集成在全样本上的“分歧”。式(8.31)的意义在于...
------------------------------------------------------------

未发现 ⚠️ Query Rewrite 丢失案例 (Bad Cases)


### 1.4 扩展场景：多轮对话的指代消解

上面演示了单轮查询改写。在实际应用中，还有一个重要场景：**多轮对话的指代消解**（例如："这个算法"指代前面提到的"决策树"）。下面演示如何处理这类场景。

#### 1.4.1 定义多轮对话改写链

与单轮改写不同，多轮对话改写需要：
- 维护**对话历史**（使用`MessagesPlaceholder`）
- 结合上下文消解指代词（"这个" → "决策树"）


In [10]:
# 定义改写prompt
contextualize_q_system_prompt = (
    "你是一个查询重写助手。你的任务是将用户最新的问题改写为一个独立的、完整的搜索查询。"
    "结合历史对话上下文，将用户问题中的指代词（如“这个”、“它”）替换为具体的指代对象。"
    "改写后的查询应该清晰、准确，适合用于文档检索。"
    "请直接输出改写后的查询，不要包含任何解释、反问或多余的文字。"
    "如果用户问题已经是独立的，则原样输出。"
)

# 构建包含对话历史的 prompt 模板
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),  # 对话历史占位符
    ("human", "{input}"),  # 当前用户输入
])

# 创建改写链：prompt -> LLM -> 输出解析器
rewrite_chain = contextualize_q_prompt | llm | StrOutputParser()

#### 1.4.2 模拟对话场景

模拟一个典型的多轮对话：
- **第一轮**: "你了解决策树吗？"
- **第二轮**: "这个算法是怎么样的？"（包含指代词）

如果直接检索第二轮查询，系统无法理解"这个算法"的含义。


In [11]:
print("=" * 80)
print("【Query改写 原理演示】")
print("=" * 80)

# 第一轮对话
print("\n第一轮对话:")
test_query = '决策树'
first_query = f"你了解{test_query}吗"
print(f"用户: {first_query}")
first_answer = """当然了他是一种机器学习方法"""
print(f"助手: {first_answer}\n")

# 构建对话历史（使用 LangChain 的消息类）
chat_history = [
    HumanMessage(content=first_query),
    AIMessage(content=first_answer)
]

# 第二轮查询（包含指代词）
second_query = "这个算法是怎么样的"
print("第二轮对话:")
print(f"用户: {second_query} (包含指代词'这个')")

【Query改写 原理演示】

第一轮对话:
用户: 你了解决策树吗
助手: 当然了他是一种机器学习方法

第二轮对话:
用户: 这个算法是怎么样的 (包含指代词'这个')


如果直接使用原始查询检索「这个算法是怎么样的」，大模型无法理解指代关系！

#### 1.4.3 执行改写并对比检索效果


In [12]:
print("\n" + "-" * 80)
print("【改写过程】")
print("-" * 80)

# 使用改写链，将对话历史和当前查询传入
rewritten_query = rewrite_chain.invoke({
    "input": second_query, 
    "chat_history": chat_history
})

print(f"原始Query: {second_query}")
print(f"改写后Query: {rewritten_query}")
print(f"\n✅ 改写成功！指代词 '这个' 被替换为具体的 '{test_query}'")


--------------------------------------------------------------------------------
【改写过程】
--------------------------------------------------------------------------------
原始Query: 这个算法是怎么样的
改写后Query: 决策树算法是怎样的

✅ 改写成功！指代词 '这个' 被替换为具体的 '决策树'


现在对比原始查询和改写后查询的检索效果：


In [23]:
print("\n" + "-" * 80)
print("【检索结果对比】")
print("-" * 80)

# 1. 使用原始query检索（包含指代词）
print("\n① 使用原始Query检索（包含指代词'这个'）:")
print(f"Query: {second_query}")
original_docs = vectorstore.similarity_search(second_query, k=3)
print(f"\n检索到 {len(original_docs)} 个文档:")
for i, doc in enumerate(original_docs, 1):
    content_preview = doc.page_content[:200].replace('\n', ' ')
    contains_keyword = "✅" if test_query in doc.page_content else "❌"
    print(f"{contains_keyword}:\t文档{i} [页码:{doc.metadata['page']}] 包含'{test_query}'")
    print(f"内容: {content_preview}...")

# 2. 使用改写后的query检索
print(f"\n② 使用改写后Query检索（明确指出'{test_query}'）:")
print(f"Query: {rewritten_query}")
rewritten_docs = vectorstore.similarity_search(rewritten_query, k=3)
print(f"\n检索到 {len(rewritten_docs)} 个文档:")
for i, doc in enumerate(rewritten_docs, 1):
    content_preview = doc.page_content[:200].replace('\n', ' ')
    contains_keyword = "✅" if test_query in doc.page_content else "❌"
    print(f"{contains_keyword}:\t文档{i} [页码:{doc.metadata['page']}] 包含'{test_query}'")
    print(f"内容: {content_preview}...")



--------------------------------------------------------------------------------
【检索结果对比】
--------------------------------------------------------------------------------

① 使用原始Query检索（包含指代词'这个'）:
Query: 这个算法是怎么样的

检索到 3 个文档:
❌:	文档1 [页码:116] 包含'决策树'
内容: ∂Pkl=1αl−1∂αi=∂(α1+α2+...+αi+...+αk−1)∂αi=1综合两项求导结果,并令导数等于零即得式(9.37)。9.4.11式(9.38)的推导注意,在西瓜书第14次印刷中式(9.38)上方的一行话进行了勘误:“两边同乘以αi,对所有混合成分求和可知λ=−m”,将原来的“样本”修改为“混合成分”。对公式(9.37)两边同时乘以αi可得mXj=1αi·p(xj|µi,Σ...
✅:	文档2 [页码:96] 包含'决策树'
内容: 外面;第一项e−αP|D|i=1D′t(xi)与h(x)无关,因此对于任意α>0,使ℓexp(Ht−1+αh|D)最小的h(x)只需要使第二项最小即可,即ht=argminh eα−e−α|D|Xi=1D′t(xi)I(f(xi)̸=h(xi))对于任意α>0,有eα−e−α>0,所以上式中与h(x)无关的正系数可以省略:ht=argminh|D|Xi=1D′t(xi)I(f(xi)̸=h(xi...
❌:	文档3 [页码:94] 包含'决策树'
内容: 8.2.13式(8.17)的推导当f(x)=h(x)时，I(f(x)̸=h(x))=0，f(x)h(x)=1，1−2I(f(x)̸=h(x))=1；当f(x)̸=h(x)时，I(f(x)̸=h(x))=1，f(x)h(x)=−1，1−2I(f(x)̸=h(x))=−1。综上，左右两式相等。8.2.14式(8.18)的推导本式基于式(8.17)的恒等关系，由式(8.16)推导而来。Ex∼Dt[f(x...

② 使用改写后Query检索（明确指出'决策树'）:
Query: 决策树算法是怎样的

检索到 3 个文档:
✅:	文档1 [页码:44] 包含'决策树

可以看出，改写可以将「这个算法是怎么样的」改写为「决策树算法是怎样的？」，检索到的内容均和决策树相关，检索效果更好。

## 第2章：HyDE（假设文档嵌入）

### 2.1 理论介绍

HyDE（Hypothetical Document Embeddings）算法由 Gao 等人于 2023 年在[《Precise Zero-Shot Dense Retrieval without Relevance Labels》](https://arxiv.org/abs/2212.10496)论文中提出，背景是为了解决零样本密集检索的挑战，结合生成模型和嵌入模型，无需相关标签即可提升检索效果。

HyDE 即假设文档嵌入，旨在通过生成假设性文档来提升检索效果。其核心思想如下：
* 首先，在用户输入查询后，使用 LLM 在没有外部知识的情况下生成一个假设的、理想化的相关文档。生成的文档不一定是真实存在的，这一步的目的是捕捉查询的深层语义意图。
* 接下来，将这个假设性文档和原始查询都转换为嵌入，用于向量检索的相似性计算。
* 然后，从目标文档数据库中检索，找到在向量空间中最接近这些嵌入的真实文档。

![HyDE](./figures/HyDE.png)

### 2.2 优势和局限性

区别于直接使用 query 来检索嵌入相似性，HyDE 是检索从一个答案到另一个答案的嵌入相似性，可以让根据 query 生成的假设性文档和目标文档的语义空间保持一致。通过生成假设文档，HyDE 能覆盖更广泛的语义关联，具备更好的泛化能力；并且无需依赖标注数据，具有零样本（Zero-Shot）适应能力。然而，HyDE 方法依赖生成模型的可靠性，若模型本身存在偏见或知识过时，最终检索结果可能会不理想。


### 2.3 代码实现


我们实现一个HyDE的检索方法，并使用通用评估函数来对比效果。

In [25]:
# 定义HyDE生成假设文档的prompt
hyde_template = """请根据以下问题给出回答。\n\n问题: {question}\n\n回答:\n"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_chain = hyde_prompt | llm | StrOutputParser()

def run_hyde(query: str, topk: int = 3):
    """
    执行HyDE检索方法
    """
    hypothetical_doc = hyde_chain.invoke({"question": query}) # HYDE 生成假设文档
    docs = vectorstore.similarity_search(hypothetical_doc, k=topk) # 检索
    return docs, hypothetical_doc

In [26]:
# 执行评估
hyde_eval_results = evaluate_retrieval_method(
    method_name="HyDE",
    test_indices=test_indices,
    qa_pairs=qa_pairs,
    vectorstore=vectorstore,
    method_func=run_hyde
)

HyDE Testing: 100%|██████████| 11/11 [03:37<00:00, 19.73s/it]


=============== 📊 评估总结 (Summary) ===============
测试样本数: 11
Baseline Recall@3: 45.5%
HyDE     Recall@3: 54.5% (提升: +9.1%)


In [33]:
# 展示优化效果
display_cases(hyde_eval_results, "HyDE", case_type="win")

# 分析副作用
display_cases(hyde_eval_results, "HyDE", case_type="loss")


✅ HyDE 优化收益 (Good Cases)
Case 1 | Index: 58 | Target: P103
【Query】: 解释8.5.13节中提到的a+bm·a+cm与am之间的区别，并阐述为什么在集成学习中，单独考虑各个分类器的概率（如a+bm·a+cm）与同时考虑它们的联合概率（如am）是重要的。
【Answer】: 在8.5.13节中，式(8.42)被拆分为更容易理解的形式，其中提到a+bm·a+cm为分类器hi与hj将样本预测为+1的概率的乘积，它表示的是分类器hi与hj独立预测的概率。而am表示的是分类器hi...
【Process】:
   在8.5.13节中提到的表达式a+bm·a+cm与am之间的区别主要在于它们代表的是不同的概率估计方式。  1. a+bm·a+cm： 这个表达式通常出现在集成学习（如随机森林、梯度提升树等）中，表示的是多个分类器（或基分类器）的预测概率之和。这里的m表示基分类器的数量，b和c是常数，它们通常是通过交叉验证等方法来确定的。具体来说，a+bm·a+cm可以理解为每个基分类器预测为正类的概率a加上一个...
【Docs (Top-3)】:
     Doc 1 (P88): 8.1个体与集成基学习器(baselearner)的概念在论文中经常出现，可留意一下；另外，本节提到的投票法有两种，除了本节的多数投票(majorityvoti...
     Doc 2 (P101): 8.5.2式(8.28)的解释¯A(h|x)=TXi=1wiA(hi|x)=TXi=1wi(hi(x)−H(x))2该式表示对各个个体学习器的“分歧”加权平均的...
  ✅ Doc 3 (P103): 8.5.13式(8.42)的解释将式(8.42)拆分为如下形式，将会很容易理解其含义：p2=a+bm·a+cm+c+dm·b+dm其中a+bm为分类器hi将样本...
------------------------------------------------------------

未发现 ⚠️ HyDE 丢失案例 (Bad Cases)


可以看出通过 HYDE 拓展了原始的检索效果。

## 第3章：Step-Back Prompting

### 3.1 理论介绍

Step-Back Prompting 由 Google DeepMind 团队于 2024 年 3 月在[《Take a Step Back: Evoking Reasoning via Abstraction in Large Language Models》](https://arxiv.org/pdf/2310.06117)论文中提出，是一种旨在提升 LLM 复杂问题推理能力的提示技术，其核心思想是引导模型进行抽象思考，从具体问题中抽象出高层次的概念或原理，从而增强对复杂问题的推理能力。具体策略如下：
* 首先使用 LLM "后退一步"生成高层次的抽象概念，将推理建立在抽象概念的基础上，以减少在中间推理步骤中出错的概率；
* 然后将抽象的概念和原始问题都用来进行检索，并把检索到的结果都用来作为 LLM 响应的基础。

![Step-Back Prompting](./figures/Step-Back-Prompting.png)

例如，上述图片中的一个例子是，"Estella Leopold 在 1954 年 8 月至 1954 年 11 月期间去了哪所学校？"，这类问题对于 LLM 来说很容易答错，因为文档中可能并没有直接相关的内容。但是如果后退一步，站在更高层次对问题进行抽象，提出一个新的问题："Estella Leopold 的教育背景是什么？"，那 LLM 可以先将 Estella Leopold 的教育经历都列出来，然后将这些信息和原始问题放在一起，这样对于 LLM 来说就可以很容易给出正确答案。

### 3.2 优势和局限性

Step-Back Prompting 通过 "抽象化->具体推理" 的分层策略，能够提升大模型在复杂任务中的表现，但其效果也高度依赖模型本身的领域知识和抽象能力。

### 3.3 代码实现

我们实现一个Step-Back Prompting的检索方法，并使用通用评估函数来对比效果。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

# 1. 定义 Few Shot 示例：使用通用的常识性示例，不针对特定数据集优化
examples = [
    {
        "input": "中国农历年腊月三十日放假吗？",
        "output": "中国农历假期时间有哪些？",
    },
    {
        "input": "李白是哪个朝代的诗人？",
        "output": "李白的生平简介?",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 2. 定义强约束的 System Prompt
system_prompt = """你是一个查询优化专家。你的任务是执行"Step-Back Prompting"（后退一步提问）。\n
**任务定义**：\n
不要直接回答用户的问题。相反，请退后一步，针对用户问题背后的**核心概念**或**基础原理**，提出了一个更抽象、更宏观的背景知识问题。\n
**严格的输出约束**：\n
1. **绝对禁止**回答用户的问题！\n
2. **绝对禁止**输出解释、分析"等内容。
3. **仅输出**一个以问号结尾的中文问题。"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        few_shot_prompt,
        ("user", "{question}"),
    ]
)

# 构建 Chain
stepback_question_gen = prompt | llm | StrOutputParser()


In [18]:
# 定义 Step-Back Prompting 的具体执行逻辑
def run_stepback(query: str):
    # 1. 生成抽象问题
    stepback_question = stepback_question_gen.invoke({"question": query})
    
    # 2. 使用抽象问题检索
    docs = vectorstore.similarity_search(stepback_question, k=3)
    
    # 返回检索结果和中间过程（抽象问题）
    return docs, f"抽象问题: {stepback_question}"

# 执行评估
stepback_eval_results = evaluate_retrieval_method(
    method_name="Step-Back",
    test_indices=test_indices,
    qa_pairs=qa_pairs,
    vectorstore=vectorstore,
    method_func=run_stepback
)

Step-Back Testing: 100%|██████████| 11/11 [00:21<00:00,  1.96s/it]


=============== 📊 评估总结 (Summary) ===============
测试样本数: 11
Baseline Recall@3: 45.5%
Step-Back Recall@3: 45.5% (提升: +0.0%)


In [34]:
# 展示优化效果
display_cases(stepback_eval_results, "Step-Back", case_type="win")

# 分析副作用
display_cases(stepback_eval_results, "Step-Back", case_type="loss")


✅ Step-Back 优化收益 (Good Cases)
Case 1 | Index: 59 | Target: P106
【Query】: 在AdaBoost算法中，如何根据指数损失函数推导出第t轮迭代的学习器ht(x)的权重αt？
【Answer】: 在AdaBoost算法中，第t轮迭代的学习器ht(x)的权重αt是通过求解指数损失函数的最小值得到的。根据8.11式的推导，αt的计算公式为：

αt = 1/2 * ln((1 - ϵt) / ϵt...
【Process】:
   抽象问题: 指数损失函数在机器学习中的作用是什么？...
【Docs (Top-3)】:
     Doc 1 (P91): 第一行到第二行显然成立，第二行到第三行是利用了argmax函数的定义。argmaxy∈{−1,1}P(f(x)=y|x)表示使得函数P(f(x)=y|x)取得最...
  ✅ Doc 2 (P106): 将负号去掉,最小化变为最大化问题ht=argmaxhXx∈DD(x) e−f(x)Ht−1(x)f(x)h(x)=argmaxhEx∼De−f(x)Ht−1...
     Doc 3 (P54): 假设此时误分类样本集合为M⊆T，对任意一个误分类样本(x,y)∈M来说，当wTx−θ⩾0时，模型输出值为ˆy=1，样本真实标记为y=0；反之，当wTx−θ<0时...
------------------------------------------------------------

⚠️ Step-Back 丢失案例 (Bad Cases)
Case 1 | Index: 109 | Target: P171
【Query】: 根据所提供的上下文信息，推导并解释式(13.15)的数学表达式，以及它是如何根据“相似的样本应具有相似的标记”的原则，帮助预测未标记样本的标记。
【Answer】: 式(13.15)的推导首先基于式(13.14)对fu求导，得到∂E(f)/∂fu的结果，并令其等于0。这里的fu指的是在未标记样本上的预测结果，而Duu、Wuu、Wul可以通过式(13.11)得到。式...
【Process】:
   抽象问题: “相似的样本应具有相似的标记”这一原则在机器学习中如何体现？...
【Docs (Top-3)】:
     Do

可以看出通过 Step-Back Prompting 同样可以拓展原始的检索效果。但同时在一些案例中，Step-Back Prompting 会丢失一些重要的信息，导致检索效果不佳。

## 第4章：子查询（Multi-Query）（MultiQueryRetriever）

### 4.1 理论介绍

子查询使用分而治之的方法来处理复杂的问题，其核心思想是在问答过程中生成并提出与主问题相关的子问题，以便更好地理解和回答主问题。这些子问题通常更具体，可以帮助系统更深入地理解主问题，从而提高检索准确性和提供正确的答案。具体步骤如下：

* 查询分解：利用大语言模型将用户的原始查询分解为多个子查询。
* 检索阶段：针对每个子查询，检索模块在知识库或文档集合中检索相关的文档。
* 合并与生成：将检索到的文档集拼接起来，再由大语言模型根据这些上下文信息生成最终答案。

### 4.2 优势和局限性
子查询通过将复杂问题分解为多个子问题，可以更精准地检索到与每个子问题相关的文档，从而提高整体检索的准确性和相关性。并且由于子查询可以覆盖原始查询的多个方面，生成的答案也更加丰富和准确。
但同时也存在一些局限性，例如过多的子查询可能增加系统的计算负担和延迟，最终的效果也依赖检索模块的性能

### 4.3 代码实现


In [ ]:


from langchain.retrievers.multi_query import MultiQueryRetriever
import logging

# 关闭 MultiQueryRetriever 的详细日志
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.WARNING)

# 基于现有向量库构建子查询检索器
retriever = vectorstore.as_retriever()
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm,
)

def run_subquery(query: str, topk: int = 3):
# 注意：为了演示生成的子查询内容，此处显式增加了一次 LLM 调用。
# 在生产环境中，可以直接调用 multi_query_retriever.invoke(query)，无需此步骤。
    # 1. 先显式调用内部的 llm_chain 生成子查询文本
    try:
        generated = multi_query_retriever.llm_chain.invoke({"question": query})
        # 不同版本返回类型可能不一样，这里统一转成字符串
        subquery_text = generated 
    except Exception as e:
        subquery_text = f"子查询生成失败: {e}"

    # 2. 再用 MultiQueryRetriever 实际做检索
    docs = multi_query_retriever.invoke(query)
    docs = docs[:topk]

    # 3. 把“子查询文本”作为 debug_info 返回，给 [中间过程] 打印用
    debug_info = subquery_text
    return docs, debug_info

In [21]:
# 在同一批测试样本上评估子查询方法
subquery_results = evaluate_retrieval_method(
    method_name="Multi-Query",
    test_indices=test_indices,
    qa_pairs=qa_pairs,
    vectorstore=vectorstore,
    method_func=lambda q: run_subquery(q, topk=3),  # 与 Baseline 的 k 保持一致
    topk=3,
)

Multi-Query Testing: 100%|██████████| 11/11 [01:50<00:00, 10.07s/it]


=============== 📊 评估总结 (Summary) ===============
测试样本数: 11
Baseline Recall@3: 45.5%
Multi-Query Recall@3: 54.5% (提升: +9.1%)


In [35]:
# 展示优化效果
display_cases(subquery_results, "Multi-Query", case_type="win")

# 分析副作用
display_cases(subquery_results, "Multi-Query", case_type="loss")


✅ Multi-Query 优化收益 (Good Cases)
Case 1 | Index: 108 | Target: P170
【Query】: 根据给定的文本内容，假设W是一个对称矩阵，证明式(13.12)中的第一项和第二项可以进行交换次序，并解释为什么可以这么做。
【Answer】: 在式(13.12)的推导中，对于能量函数E(f)的展开式如下：
\[ E(f) = \frac{1}{2} \sum_{i=1}^{m} \sum_{j=1}^{m} (W)_{ij}(f(xi) -...
【Process】:
   1. 请证明在给定文本内容中，对称矩阵W下，式(13.12)的第一项和第二项可以互换顺序，并阐述其互换的合理性。...
   2. 针对文本中的对称矩阵W，探讨式(13.12)中的前两项是否可以交换位置，并给出交换的理由。...
   3. 在文本所述的对称矩阵W情境下，分析式(13.12)中的第一项与第二项能否调换顺序，并解释为何这种调换是可行的。...
【Docs (Top-3)】:
     Doc 1 (P174): 可以验证,上式的矩阵A=S◦FF⊤元素之和Pmi,j=1(A)ij等于tr S⊤FF⊤,这是因为tr(S)11(S)12···(S...
     Doc 2 (P173): 个和求号Pmi=1Pmi=1的形式,并将上面拆分的四项中的前两项代入,得mXi,j=1(W)ij1diFiF⊤i=mXi=11diFiF⊤imXj=1(W)ij...
  ✅ Doc 3 (P170): 中的(f(xi)−f(xj))2相应地越小越好(即“相似的样本应具有相似的标记”),如此才能达到能量函数E(f)越小的目的。首先对式(13.12)的第1行式子进...
------------------------------------------------------------

未发现 ⚠️ Multi-Query 丢失案例 (Bad Cases)


可以看出 Multi-Query 在检索效果上也有不错的表现。

## 第5章：总结与对比

#### 5.1 方法优劣势对比

| 方法       | 优势                             | 劣势                             | 适用场景                     | 延迟/成本 |
|------------|----------------------------------|----------------------------------|------------------------------|-----------|
| 直接检索   | 速度快，无额外LLM调用            | 语义匹配弱，易遗漏               | 关键词匹配、查询明确         | 极低      |
| Query改写  | 纠正输入、消解指代、语义对齐     | 依赖LLM，可能改错                | 多轮对话、口语化输入         | 低        |
| HyDE       | 零样本适应强，语义对齐好         | 幻觉风险，生成慢                 | 知识密集型、无监督场景       | 高        |
| Step-Back  | 提升泛化，减少过拟合             | 易丢失关键实体/数字              | 复杂推理、宏观原理问题       | 低        |
| 子查询     | 覆盖面广，多角度检索             | 检索多次，耗时增加               | 复杂多维度、歧义问题         | 中        |

#### 5.2 总结

在实际场景中，Query对齐是性价比极高的优化手段。它无需重构模型、索引或高昂微调成本，仅通过优化用户输入，就能显著提升检索与生成效果。​
Query对齐不仅实现语义对齐，更能解决“脏输入”问题——面对口语化表述、指代歧义、错漏等问题，它可通过纠错、消歧、补全，将输入转化为标准化Query，从源头降低检索偏差。​
但Query对齐并非万能，需结合场景选优化手段：查询明确选直接检索（低延迟）；知识密集无标注数据用HyDE（零样本优势）；复杂推理问题靠Step-Back（提升泛化）；多维度查询则适合子查询（覆盖全面）。​
综上，实际应用需以Query对齐为基础，再适配场景选择补充优化手段，通过需求匹配实现效果高效提升。